In [4]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn 
seaborn.set(font_scale=1.5,style='ticks')
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions

In [314]:
n_classes = 2
nB = 3
ndat = 500
beta = np.random.normal(size=[n_classes,nB])


X = np.random.normal(size=[nB,ndat])

softmax = lambda b,x: np.exp(b.dot(x))/(np.sum(np.exp(b.dot(x)),axis=0))


pvals = softmax(beta,X)

y = np.zeros([n_classes,ndat])
for kk,i in enumerate(pvals.T):
    y[:,kk] = np.random.multinomial(n=1,pvals=i)

In [315]:

pvals = softmax(beta,X)

In [316]:
tf.nn.softmax(beta.dot(X).T).numpy()[0]

array([0.02201189, 0.97798811])

In [317]:
y = np.zeros([n_classes,ndat])
for kk,i in enumerate(pvals.T):
    y[:,kk] = np.random.multinomial(n=1,pvals=i)

In [318]:
class Bernoulli(tfp.glm.ExponentialFamily):
  """`Bernoulli(probs=mean)` where `mean = sigmoid(matmul(X, weights))`."""

  _is_canonical = True

  def _call(self, r):
    mean = tf.nn.sigmoid(r)
    variance = grad_mean = mean * tf.nn.sigmoid(-r)
    return mean, variance, grad_mean

  def _log_prob(self, y, r):
    return tfd.Bernoulli(logits=r).log_prob(y)

In [319]:
sigmoid = lambda x: 1/(1+np.exp(-x))

In [320]:
y

array([[0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1.,
        0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0.,
        1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0.,
        1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0.,
        1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1.,
        0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0.,
        0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
        0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
        0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1.,
        0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
        1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 

In [321]:
sigmoid(2)

0.8807970779778823

In [285]:
sigmoid(-2)

0.11920292202211755

In [348]:
class MtlGLM(tfp.glm.ExponentialFamily):
    """`Bernoulli(probs=mean)` where `mean = sigmoid(matmul(X, weights))`."""

    _is_canonical = False

    def _call(self, r):
        mean = tf.nn.softmax(r)
        grad_mean = mean * (1-mean)
        variance = mean[None,:]*mean[:,None]
        variance[np.diag_indices_from(variance)] = mean * (1-mean)
        return mean, variance, grad_mean

    def _log_prob(self, y, r):
        #print(len(y))
        return tf.reduce_mean(tfd.Multinomial(1,probs=r).log_prob(y))

In [349]:
tfd.Multinomial(1,logits=[0,.5,.5]).log_prob(np.array([0,1,0]).reshape(1,-1))

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-0.9580201], dtype=float32)>

In [350]:
glm = MtlGLM()

In [351]:
coefs = np.zeros_like(beta)

In [352]:
coefs

array([[0., 0., 0.],
       [0., 0., 0.]])

In [353]:
beta.shape

(2, 3)

In [354]:
coefs = np.zeros_like(beta)


In [355]:
#coefs = np.zeros_like(beta)
#glm = MtlGLM()
coefs,_ = tfp.glm.fit_one_step(X.T,y,glm,model_coefficients_start=coefs)

ValueError: All dimensions of input must be of equal length